<div align='center'><font size="5" color='#353B47'>Image Segmentation</font></div>
<div align='center'><font size="4" color="#353B47">Using U-Net with Keras</font></div>
<br>
<hr>

<div id="chap2"><h1 style="color:white;background:#5963ab;border-radius:5px;padding:30px;font-family:'Arial', cursive;font-size:50px;text-align:center">Setup</h1></div>

In [ ]:
import tensorflow as tf
import zipfile
import numpy as np
import random
import os
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import os
from skimage.io import imread
from skimage.transform import resize
from tqdm import tqdm

# Set the random seed correctly
SEED = 42
np.random.seed(SEED)

# Paths for the dataset (no need to unzip)
TRAIN_PATH = '/kaggle/input/dataset3/data/images'  # Adjust this path if needed
TEST_PATH = '/kaggle/input/dataset3/data/labels'   # Adjust this path if needed

# Image specifications
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

# Get list of all subfolders for train and test
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

# Define placeholders for training data
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool_)  # Changed to np.bool_

# Loop through all training IDs to read images and masks
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):  
    # Get the path to the image and mask directories
    path = os.path.join(TRAIN_PATH, id_)  # Adjust path correctly
    
    # Read and resize the image
    img = imread(path + '/images/' + id_ + '.tif')[:, :, :IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    
    # Store the processed image in the X_train array
    X_train[n] = img
    
    # Initialize an empty mask
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool_)  # Changed to np.bool_
    
    # Loop through each file in the masks directory
    for mask_file in next(os.walk(path + '/masks/'))[2]:  
        # Read and resize the mask, adding an extra dimension
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)
        
        # Merge the masks
        mask = np.maximum(mask, mask_)
    
    # Store the processed mask in the Y_train array
    Y_train[n] = mask


In [ ]:
# Create an empty array for test images with dimensions as (number of test images, height, width, channels)
# The datatype for the array is uint8 which can hold values from 0 to 255
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

# Initialize an empty list to store the original sizes of the test images
sizes_test = []

# Loop through all the test_ids
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    
    # Get the path to the test image
    path = TEST_PATH + id_
    
    # Read the image and keep the first IMG_CHANNELS channels
    img = imread(path + '/images/' + id_ + '.tif')[:, :, :IMG_CHANNELS]
    
    # Append the original size of the image to sizes_test list
    sizes_test.append([img.shape[0], img.shape[1]])
    
    # Resize the image to IMG_HEIGHT x IMG_WIDTH while keeping the pixel values
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    
    # Store the processed image into the X_test array at index n
    X_test[n] = img

# <div id="chap4"><h1 style="color:white;background:#5963ab;border-radius:5px;padding:30px;font-family:'Arial', cursive;font-size:50px;text-align:center">Training Unet</h1></div>

In [ ]:
# Create a function for a convolution block
def conv_block(inputs, num_filters):
    x = tf.keras.layers.Conv2D(num_filters, (3, 3), activation="relu", 
                               kernel_initializer="he_normal", padding="same")(inputs)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Conv2D(num_filters, (3, 3), activation="relu", 
                               kernel_initializer="he_normal", padding="same")(x)
    return x

# Create a function for the expanding path
def upsample_block(inputs, conv_prev, num_filters):
    up = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding="same")(inputs)
    concat = tf.keras.layers.concatenate([up, conv_prev])
    conv = conv_block(concat, num_filters)
    return conv

In [ ]:
# Inputs
inputs = tf.keras.layers.Input((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))

# Normalization
s = tf.keras.layers.Lambda(lambda x: x/255.0)(inputs) 

# Contraction path
c1 = conv_block(s, 16)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = conv_block(p1, 32)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = conv_block(p2, 64)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = conv_block(p3, 128)
p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)

c5 = conv_block(p4, 256)

# Expansive path
c6 = upsample_block(c5, c4, 128)
c7 = upsample_block(c6, c3, 64)
c8 = upsample_block(c7, c2, 32)
c9 = upsample_block(c8, c1, 16)

# Output layer
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

In [ ]:
# Model
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

# Compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

In [ ]:
# Callbacks
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.keras', verbose=1, save_best_only=True)  # Changed to .keras
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]


In [ ]:
results = model.fit(
    X_train, 
    Y_train, 
    validation_split = 0.1, 
    batch_size = 16, 
    epochs = 25, 
    callbacks = callbacks)

# <div id="chap5"><h1 style="color:white;background:#5963ab;border-radius:5px;padding:30px;font-family:'Arial', cursive;font-size:50px;text-align:center">Inference</h1></div>

In [ ]:
# Predictions
idx = random.randint(0, len(X_train))

preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose = 1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose = 1)
preds_test = model.predict(X_test, verbose = 1)

preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [ ]:
# Function to display images
def display_images(image):
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [ ]:
# Select a random index from training set
ix = random.randint(0, len(preds_train_t))

# Display the input image
print("Input Image:")
display_images(X_train[ix])

# Display the ground truth mask
print("Ground Truth Mask:")
display_images(np.squeeze(Y_train[ix]))

# Display the predicted mask
print("Predicted Mask:")
display_images(np.squeeze(preds_train_t[ix]))

In [ ]:
# Select a random index from validation set
ix = random.randint(0, len(preds_val_t))

# Calculate the index from where validation data starts
val_data_start = int(X_train.shape[0] * 0.9)

# Display the validation input image
print("Validation Input Image:")
display_images(X_train[val_data_start:][ix])

# Display the validation ground truth mask
print("Validation Ground Truth Mask:")
display_images(np.squeeze(Y_train[val_data_start:][ix]))

# Display the validation predicted mask
print("Validation Predicted Mask:")
display_images(np.squeeze(preds_val_t[ix]))

In [ ]:
from flask import Flask, render_template, request
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

app = Flask(__name__)


model=load_model('model.keras')


model.make_predict_function()

def predict_label(img_path):
        i = image.load_img(img_path, target_size=(28,28), grayscale=True)
        i = image.img_to_array(i)
        resize_image= np.array([i], order='C')
        img_out = image.array_to_img(resize_image.reshape(28,28,1))
        img_out.save(img_path)
        p = model.predict(resize_image)
        return p.argmax()

# routes
@app.route("/", methods=['GET', 'POST'])
def main():
    return render_template("index.html")

@app.route("/about")
def about_page():
    return "My project "

@app.route("/submit", methods = ['GET', 'POST'])
def get_output():
        if request.method == 'POST':
                img = request.files['my_image']

                img_path = "data/" + img.filename	
                img.save(img_path)

                p = predict_label(img_path)

        return render_template("index.html", prediction = p, img_path = img_path)
